<a href="https://colab.research.google.com/github/mfmceneaney/DIRC_COLAB/blob/master/DIRC_CNN_Architecture_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Using GPU speeds up CNN training
# Go to Edit > Notebook Settings > Hardware Accelerator and select GPU

In [0]:
print(" > Installing uproot...")
!pip install uproot

In [0]:
import uproot

from os import chdir, getcwd, mkdir, listdir, rmdir
from shutil import rmtree

# Standard scientific Python imports
import matplotlib.pyplot as plt
import numpy as np
import math

from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121, ResNet50V2, VGG16, VGG19, MobileNet, MobileNetV2, NASNetLarge, NASNetMobile
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# initialize images
image_kplus = np.zeros(shape=(48,144))
image_piplus = np.zeros(shape=(48,144))
images_kplus = []
images_piplus = []
images_flat_kplus = []
images_flat_piplus = []

#h2_kplus = ROOT.TH2F("kplus","Hit Pattern K+; PixelRow; PixelCol",144,-0.5,143.5,48,-0.5,47.5)
#h2_piplus = ROOT.TH2F("piplus","Hit Pattern pi+; PixelRow; PixelCol",144,-0.5,143.5,48,-0.5,47.5)

# get data from files
print("Opening files")
piplus = uproot.open("/content/gdrive/My Drive/piplus_p3_theta4_flat.root")["dircml_flat"]
kplus = uproot.open("/content/gdrive/My Drive/kplus_p3_theta4_flat.root")["dircml_flat"]

# fill histogram with # photon hits
#plt.axis([0, 250, 0, 5000])
#plt.hist(NPixelsArr, 4000) #, 0, 100) #bins='auto')
#plt.title("Histogram with 'auto' bins")
#plt.show()

eventMax = 10000
eventCounter = 0

print("Filling image arrays")
# loop over kplus events
for (PixelTimes,PixelRows,PixelCols) in zip(kplus.array("PixelTime"),kplus.array("PixelRow"),kplus.array("PixelCol")):
    #print("event")
    image_single = np.zeros(shape=(48,144))
    
    # loop over pixels within event
    for (PixelTime,PixelRow,PixelCol) in zip(PixelTimes,PixelRows,PixelCols):
        #print("PixelTime,Row,Col = %f,%d,%d" % (PixelTime,PixelRow,PixelCol))
        image_kplus[PixelCol,PixelRow] += 1
        image_single[PixelCol,PixelRow] = PixelTime
        #h2_kplus.Fill(PixelRow,PixelCol)
    
    # after each event
    images_kplus.append(image_single)
    images_flat_kplus.append(np.reshape(image_single, 6912))
    
    eventCounter = eventCounter+1
    if eventCounter > eventMax:
        break

eventCounter = 0
        
# loop over piplus events
for (PixelTimes,PixelRows,PixelCols) in zip(piplus.array("PixelTime"),piplus.array("PixelRow"),piplus.array("PixelCol")):
    #print("event")
    image_single = np.zeros(shape=(48,144))
    
    # loop over pixels within event
    for (PixelTime,PixelRow,PixelCol) in zip(PixelTimes,PixelRows,PixelCols):
        #print("PixelTime,Row,Col = %f,%d,%d" % (PixelTime,PixelRow,PixelCol))
        image_piplus[PixelCol,PixelRow] += 1
        image_single[PixelCol,PixelRow] = PixelTime
        #h2_piplus.Fill(PixelRow,PixelCol)
    
    # after each event
    images_piplus.append(image_single)
    images_flat_piplus.append(np.reshape(image_single, 6912))
    
    eventCounter = eventCounter+1
    if eventCounter > eventMax:
        break

In [0]:
# draw cumulative image
plt.axis([-0.5, 143.5, -0.5, 47.5])
plt.subplot(2, 1, 1)
plt.title("Hit Pattern Histogram K+ (top) and Pi+ (bottom)")
plt.imshow(image_kplus, cmap='viridis')
plt.subplot(2, 1, 2)
plt.title("")
plt.imshow(image_piplus, cmap='viridis')
plt.show()

# draw single event image
plt.subplot(2, 1, 1)
plt.title("Hit Time Histogram K+ (top) and Pi+ (bottom)")
plt.imshow(images_kplus[0], cmap='viridis')
plt.subplot(2, 1, 2)
plt.title("")
plt.imshow(images_piplus[0], cmap='viridis')
#plt.show()

# draw ROOT image
#can = ROOT.TCanvas("cc","cc",600,500)
#can.Divide(1,2)
#can.cd(1)
#h2_kplus.Draw("colz")
#can.cd(2)
#h2_piplus.Draw("colz")
#can.Print("HitPattern.png")

In [0]:
# Split samples into training and testing
print("Splitting sample into training and testing subsets")
split = np.array((80,10,10)) # Percentages into which to split data (% train, % val, % test)
n_kplus = len(images_kplus)
n_piplus = len(images_piplus)
n_total = n_kplus + n_piplus
print(n_kplus / split)

print(n_kplus // split)
print(n_piplus // split)
images_train_kplus = images_kplus[:int(n_kplus * split[0] / 100)]
images_train_piplus = images_piplus[:int(n_piplus * split[0] / 100)]
n_train_kplus = len(images_train_kplus)
n_train_piplus = len(images_train_piplus)
n_train_total = n_train_kplus + n_train_piplus

images_val_kplus = images_kplus[int(n_kplus * split[0] / 100):int(n_piplus * (split[0] + split[1]) / 100)]
images_val_piplus = images_piplus[int(n_piplus * split[0] / 100):int(n_piplus * (split[0] + split[1]) / 100)]
n_val_kplus = len(images_val_kplus)
n_val_piplus = len(images_val_piplus)
n_val_total = n_val_kplus + n_val_piplus

images_test_kplus = images_kplus[int(n_piplus * (split[0] + split[1]) / 100):]
images_test_piplus = images_piplus[int(n_piplus * (split[0] + split[1]) / 100):]
n_test_kplus = len(images_test_kplus)
n_test_piplus = len(images_test_piplus)
n_test_total = n_test_kplus + n_test_piplus

data_train = np.concatenate((images_train_kplus, images_train_piplus))
data_val = np.concatenate((images_val_kplus, images_val_piplus))
data_test = np.concatenate((images_test_kplus, images_test_piplus))
data = np.concatenate((images_kplus, images_piplus))

target_train = np.concatenate((np.ones(n_train_kplus), np.zeros(n_train_piplus)))
target_val = np.concatenate((np.ones(n_val_kplus), np.zeros(n_val_piplus)))
target_test = np.concatenate((np.ones(n_test_kplus), np.zeros(n_test_piplus)))
target = np.concatenate((np.ones(n_kplus), np.zeros(n_piplus)))


print(data_train.shape)
print(data.shape)
print(target.shape)
print(np.unique(target))

print("Training size = %d" % len(data_train))
print("Testing size = %d" % len(data_test))
"""
print(images_kplus[1].shape)
print(images_piplus[0].shape)

print(data.max())
print(data_train.max())
print(data_test.max())
print(data.min())
print(data_train.min())
print(data_test.min())
"""

In [0]:
# Reshape Data to have a single channel
data_train_images = data_train.reshape(len(data_train), 48, 144, 1)
target_train_images = target_train.reshape(len(target_train))

data_val_images = data_val.reshape(len(data_val), 48, 144, 1)
target_val_images = target_val.reshape(len(target_val))

data_test_images = data_test.reshape(len(data_test), 48, 144, 1)
target_test_images = target_test.reshape(len(target_test))

# Normalize pixel absolute values to be between 0 and 1
data_train_images = data_train_images / data.max()
data_val_images = data_val_images / data.max()
data_test_images = data_test_images / data.max()

#print(data_train_images.max())
#print(data_test_images.max())

print("Data shape: "+str(data_train_images.shape))
print("Target shape: "+str(target_train_images.shape))

In [0]:
###################################
# Evaluation function definitions #
# Matthew McEneaney               #
# 1/12/20                         #
###################################

def write_summary(model,name,verbosity):
  '''
  Write model architecture summary into a .txt file.
  '''

  def myFunc(text):
    outF.write(text+'\n')
  
  # Write summary to file
  outF = open(name+"_summary.txt", "w")
  model.summary(print_fn=myFunc)
  outF.close()
  if verbosity == 1:
    model.summary()

def plot_metrics(name,batch_size,results,history,n,verbosity):
  '''
  Plot training metrics (accuracy, validation accuracy, 
  loss, validation, loss) as a function of training 
  epoch and save in .png and .root file formats.
  '''

  # Plot as .png
  f = plt.figure(figsize=(16,10))
  val = plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'],'--', color=val[0].get_color())
  val = plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'],'--', color=val[0].get_color())
  plt.title('Model: '+name+' '+str(n)+' metrics, batch_size = '+str(batch_size))
  plt.ylabel('Arbitrary Units')
  plt.xlabel('Epoch')
  plt.legend(['accuracy', 'val_accuracy','loss', 'val_loss'], loc='best')
  if verbosity == 1:
    plt.show()
  f.savefig("epochs_metrics"+str(n)+".png")

  # Convert accuracy and loss plots into ROOT histograms
  if verbosity == 1:
    print("Converting training epochs plots to ROOT...")

  epochs = np.array([i for i in range(len(history.history['accuracy']))])

  file = uproot.recreate('epochs_accuracy_'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["acc"] = np.histogram2d(history.history['accuracy'],epochs)

  file = uproot.recreate('epochs_val_accuracy'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["val_acc"] = np.histogram2d(history.history['val_accuracy'],epochs)

  file = uproot.recreate('epochs_loss'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["loss"] = np.histogram2d(history.history['loss'],epochs)

  file = uproot.recreate('epochs_val_loss'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["val_loss"] = np.histogram2d(history.history['val_loss'],epochs)

def plot_stability(metrics,name,batch_size,verbosity):
  '''
  Plot the test metrics (accuracy, loss, chi2) recorded for 
  several model iterations in both .png and .root file formats.
  '''

  if verbosity == 1:
    print("Plotting purity and \u03C7\u00b2")

  # Plot accuracy
  f = plt.figure()
  plt.plot(metrics[0],metrics[1],'b.', scaley = False)
  plt.title("Model: "+name+", Accuracy, batch_size = "+str(batch_size))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_acc.png")

  # Plot loss
  f = plt.figure()
  plt.plot(metrics[0],metrics[2],'b.', scaley = False)
  plt.title("Model: "+name+", Loss, batch_size = "+str(batch_size))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_loss.png")

  # Plot kaon chi2
  f = plt.figure()
  plt.plot(metrics[0],metrics[3][0],'b.')
  plt.title("Model: "+name+", Kaon \u03C7\u00b2, batch_size = "+str(batch_size))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_chi2_kaon.png")

  # Plot pion chi2
  f = plt.figure()
  plt.plot(metrics[0],metrics[3][1],'b.')
  plt.title("Model: "+name+", Pion \u03C7\u00b2, batch_size = "+str(batch_size))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_chi2_pion.png")

  # Plot total chi2
  f = plt.figure()
  plt.plot(metrics[0],metrics[3][2],'b.')
  plt.title("Model: "+name+", Total \u03C7\u00b2, Layers = 32, batch_size = "+str(batch_size))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_chi2_tot.png")
    
  # Convert accuracy and loss plots into ROOT histograms
  if verbosity == 1:
    print("Converting stability plots to ROOT...")

  file = uproot.recreate("stability_acc.root", compression=uproot.ZLIB(4))
  file["acc"] = np.histogram2d(metrics[0],metrics[1])

  file = uproot.recreate("stability_loss.root", compression=uproot.ZLIB(4))
  file["loss"] = np.histogram2d(metrics[0],metrics[2])

  file = uproot.recreate("stability_chi2_kaon.root", compression=uproot.ZLIB(4))
  file["chi2k"] = np.histogram2d(metrics[0],metrics[3][0])

  file = uproot.recreate("stability_chi2_pion.root", compression=uproot.ZLIB(4))
  file["chi2p"] = np.histogram2d(metrics[0],metrics[3][1])

  file = uproot.recreate("stability_chi2_tot.root", compression=uproot.ZLIB(4))
  file["chi2t"] = np.histogram2d(metrics[0],metrics[3][2])

def evaluate_model(model,metrics,n,verbosity):

  '''
  Record the decisions of the CNN (kaon or pion) and 
  test metrics (accuracy and loss) for a single training
  iteration. Also calculate and record the chi2 and error
  for kaon, pion decisions and all decisions respectively.
  '''

  # Keep track of results for stability plots
  metrics[0].append(n)
  metrics[1].append(results[1])
  metrics[2].append(results[0])

  # Evaluate performance by comparing test and training classifier (model) response
  decisions = []
  for (X,y) in ((data_train_images, target_train_images), (data_test_images, target_test_images)):
      d1 = model.predict_proba(X[y>0.5])[:,1].ravel()
      d2 = model.predict_proba(X[y<0.5])[:,1].ravel()
      decisions += [d1, d2]
      
  print("Filling histograms...")

  bins = 100
  low = min(np.min(d) for d in decisions)
  high = max(np.max(d) for d in decisions)
  low_high = (low,high)
  f = plt.figure()
  plt.clf()
  # Plot training decisions
  plt.hist(decisions[0], color='r', alpha=0.5, range=low_high, bins=bins, histtype='stepfilled', density=True, label='Kaon (train)')
  plt.hist(decisions[1], color='b', alpha=0.5, range=low_high, bins=bins, histtype='stepfilled', density=True, label='Pion (train)')

  # make histogram and get error bars
  hist, bins = np.histogram(decisions[2], bins=bins, range=low_high, density=True)
  width = (bins[1] - bins[0])
  center = (bins[:-1] + bins[1:]) / 2

  # plot histogram for signal test sample (Kaon Test)
  scale = len(decisions[2]) / sum(hist)
  err = np.sqrt(hist * scale) / scale
  plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='Kaon (test)')

  # make and plot histogram for background test sample (Pion Test)
  hist, bins = np.histogram(decisions[3], bins=bins, range=low_high, density=True)
  scale = len(decisions[2]) / sum(hist)
  err = np.sqrt(hist * scale) / scale
  plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='Pion (test)')

  plt.xlabel('Classifier output')
  plt.ylabel('Arbitrary units')
  plt.legend(loc='best')
  f.savefig('classifier_'+str(n)+'_output.png')
  if verbosity == 1:
    plt.show()

  # Create test and train decisions histograms for kaon
  hist_kaon_test, bins = np.histogram( decisions[0], bins=bins, range=low_high, density=False)
  hist_kaon_train, bins = np.histogram( decisions[2], bins=bins, range=low_high, density=False)
  
  # Find chi2 values and error for kaon
  hist_kaon_chi2 = np.nan_to_num(np.divide(np.square(np.subtract(hist_kaon_test, hist_kaon_train)),np.add(hist_kaon_test,hist_kaon_train)))
  scale = len(decisions[2]) / sum(hist_kaon_chi2)
  err_kaon = np.sqrt(hist_kaon_chi2 * scale) / scale
  
  # Create test and train decisions histograms for pion
  hist_pion_test, bins = np.histogram( decisions[1], bins=bins, range=low_high, density=False)
  hist_pion_train, bins = np.histogram( decisions[3], bins=bins, range=low_high, density=False)
  
  # Find chi2 values and error for pion
  hist_pion_chi2 = np.nan_to_num(np.divide(np.square(np.subtract(hist_pion_test, hist_pion_train)),np.add(hist_pion_test,hist_pion_train)))
  scale = len(decisions[2]) / sum(hist_pion_chi2)
  err_pion = np.sqrt(hist_pion_chi2 * scale) / scale
  
  # Find chi2 for kaon and pion and total
  chi2_kaon = sum(hist_kaon_chi2)
  chi2_pion = sum(hist_pion_chi2)
  chi2_total = chi2_kaon + chi2_pion
  err_total = math.sqrt(sum(err_kaon**2 + err_pion**2))

  # Record metrics values
  metrics[3][0].append(chi2_kaon)
  metrics[3][1].append(chi2_pion)
  metrics[3][2].append(chi2_total)
  metrics[4][0].append(err_kaon)
  metrics[4][1].append(err_pion)
  metrics[4][2].append(err_total)

def evaluate_architecture(model,name,opt=Adam(),batch_size=320,epochs=100,iterations=10,verbosity=1):

  '''
  Create a designated folder in which to place all the files for the evaluation of a given architecture.
  Record the architecture of the given architecture and plot and record the training metrics (accuracy, 
  validation accuracy, loss, validation loss) as a function of the training epoch for the given number 
  of epochs (default 100) with a given batch size (default 320).  Also, evaluate the stability of the 
  architecture test metrics (accuracy and loss) over a given number of model iterations (default 10).
  '''

  # Internal variable definitions
  metrics = [ [], [], [],  [ [], [], [] ],  [ [], [], [] ]  ]
  minimum = 1
  maximum = iterations + 1
  step = 1
  executable = False

  # Create file directory
  chdir('/content/gdrive/My Drive/')
  for entry in listdir():
    if entry == name:
      while True:
        resp = str(input('This file already exists.  Do you wish to continue? (y/n):'))
        if resp == 'y':
          rmtree(name)
          executable = True
          break
        elif resp == 'n':
          executable = False
          break
      break
    
  if executable == True:
    # Make file directory
    mkdir(name)
    chdir(name)

    # Record model architecture
    write_summary(model,name,verbosity)

    # Train architecture several times to assess stability    
    for n in range(minimum,maximum,step):
      if verbosity == 1:
        print('###################### \
        # Creating model '+str((n - minimum + 1))+'... #\n \
        ######################')
      
      model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=opt,
                    metrics=['accuracy'])

      history = model.fit(data_train_images,
                          target_train_images,
                          epochs=numEpochs,
                          verbose=verbosity,
                          batch_size=batch_size, 
                          validation_data=(data_val_images,target_val_images))
      
      results = model.evaluate(data_test_images,
                               target_test_images,
                               batch_size=batch_size,
                               verbose=verbosity)
      
      # Plot trainging metrics as a function of epoch
      plot_metrics(name,batch_size,results,history,n,verbosity)

      # Evaluate test accuracy, loss, and chi2 for model iteration
      evaluate_model(model,metrics,results,n,verbosity)

    # Plot test accuracy, loss, and chi2 as a function of model iteration
    plot_stability(metrics,name,batch_size,verbosity)

    # Go back up to home directory
    if verbosity == 1:
      listdir()
    chdir("..")
    if verbosity == 1:
      getcwd()
      print("\nDone\n")


In [0]:
#####################
# Model Definitions #
# Matthew McEneaney #
# 1/12/20           #
#####################

############################################### TensorFlow Example Architecture ###########################################
# check out https://www.tensorflow.org/tutorials/images/cnn
def tf_cnn_example(num_classes):
  # CNN takes 3 dimensional tensor input (image height, image width, color channel)
  model = models.Sequential()
  # Padding='same' significantly improves results
  # model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
  model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), strides=(1,1), activation='relu'))
  model.add(layers.Flatten())
  # model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64, activation='relu'))
  # model.add(layers.Dropout(0.1))
  model.add(layers.Dense(num_classes, activation='softmax'))

  return model

################################### Simple CNN for CIFAR-10 Dataset, use Adam Optimizer #######################################
# Check out https://keras.io/examples/cifar10_resnet/ for examples of this architecture and others
def tf_cifar_example(num_classes):
  model = models.Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=(48, 144, 1)))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  return model

  # # Example used this optimizer
  # initiate RMSprop optimizer
  # opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

####################################### Try Bottleneck Layers ############################################
def tf_cifar_bottleneck(num_classes):
  model = models.Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=(48, 144, 1)))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (1, 1))) # Bottleneck layers
  model.add(Conv2D(64, (3, 3))) # ...
  model.add(Conv2D(64, (1, 1))) # ...
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
  return model

#################################### Try Factorization Layers #########################################
def tf_cifar_factorization(num_classes):
  model = models.Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=(48, 144, 1)))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (1, 3))) # Factorization layers
  model.add(Conv2D(64, (3, 1))) # ...
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (1, 3))) # Factorization layers
  model.add(Conv2D(64, (3, 1))) # ...
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  return model
#####################################################################################################################

In [0]:
myModels = [[tf_cnn_example(2),'tf_cnn_example'],[tf_cifar_example(2),'tf_cifar_example'],[tf_cifar_factorization(2),'tf_cifar_factorization']]
# myModels = [[tf_cifar_factorization(2),'tf_cifar_factorization']]
for entry in myModels:
  evaluate_architecture(entry[0],entry[1])